### Importando bibliotecas necessárias

In [1]:
import requests
import pandas as pd
import pandas_profiling
import pprint
import json
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
#from gspread_pandas import Spread, Client

In [3]:
pp = pprint.PrettyPrinter(indent=4)

In [4]:
pd.set_option('display.max_colwidth',100)

## Processo de ETL

#### Função que recupera a lista de todos os deputados

In [5]:
def listaDeputados(url):
    headers = {'accept': 'application/json'}
    r = requests.get(url, headers=headers)
    return r.json()['dados']

In [6]:
listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome')

[{'id': 204554,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
  'nome': 'ABÍLIO SANTANA',
  'siglaPartido': 'PL',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
  'siglaUf': 'BA',
  'idLegislatura': 56,
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg'},
 {'id': 204521,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204521',
  'nome': 'ABOU ANNI',
  'siglaPartido': 'PSL',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36837',
  'siglaUf': 'SP',
  'idLegislatura': 56,
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204521.jpg'},
 {'id': 204379,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204379',
  'nome': 'ACÁCIO FAVACHO',
  'siglaPartido': 'PROS',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36763',
  'siglaUf': 'AP',
  'idLegislatura': 56,
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204379.j

#### Carrega lista de deputados em um dataframe pandas

In [7]:
deputados = pd.DataFrame(listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome'))

In [8]:
deputados.head()

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,204554,56,ABÍLIO SANTANA,PL,BA,https://dadosabertos.camara.leg.br/api/v2/deputados/204554,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,https://www.camara.leg.br/internet/deputado/bandep/204554.jpg
1,204521,56,ABOU ANNI,PSL,SP,https://dadosabertos.camara.leg.br/api/v2/deputados/204521,https://dadosabertos.camara.leg.br/api/v2/partidos/36837,https://www.camara.leg.br/internet/deputado/bandep/204521.jpg
2,204379,56,ACÁCIO FAVACHO,PROS,AP,https://dadosabertos.camara.leg.br/api/v2/deputados/204379,https://dadosabertos.camara.leg.br/api/v2/partidos/36763,https://www.camara.leg.br/internet/deputado/bandep/204379.jpg
3,204560,56,ADOLFO VIANA,PSDB,BA,https://dadosabertos.camara.leg.br/api/v2/deputados/204560,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,https://www.camara.leg.br/internet/deputado/bandep/204560.jpg
4,204528,56,ADRIANA VENTURA,NOVO,SP,https://dadosabertos.camara.leg.br/api/v2/deputados/204528,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,https://www.camara.leg.br/internet/deputado/bandep/204528.jpg


In [7]:
deputados.loc[deputados['nome'].str.contains('Tabata.*', regex=True, case=False)]['uri']

473    https://dadosabertos.camara.leg.br/api/v2/deputados/204534
Name: uri, dtype: object

#### Função que recupera para cada deputado todas as despesas do ano de 2019, 

In [9]:
despesas = {}
list_dataframes = []
for index, row in deputados.head(n=5).iterrows():
    pagina = 1
    while True:
        print(pagina)
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{row['id']}/despesas?ano=2019&pagina={pagina}&itens=100&ordem=ASC&ordenarPor=dataDocumento"
        print(url)
        headers = {'accept': 'application/json'}
        r = requests.get(url, headers=headers)
        if not r.json()['dados']:
            break
        desp_dep = pd.DataFrame(r.json()['dados'])
        #desp_dep[['dep_id','dep_nome','dep_partido','dep_uf']] = row[['id','nome','siglaPartido','siglaUf']]
        desp_dep['dep_id'] = row['id']
        desp_dep['dep_nome'] = row['nome']
        desp_dep['dep_partido'] = row['siglaPartido']
        desp_dep['dep_uf'] = row['siglaUf']
        #display(desp_dep.head())
        list_dataframes.append(desp_dep)
        #print(list_dataframes)
        #print(type(despesas[row['id']]))
        pagina = pagina + 1
despesas = pd.concat(list_dataframes)

1
https://dadosabertos.camara.leg.br/api/v2/deputados/204554/despesas?ano=2019&pagina=1&itens=100&ordem=ASC&ordenarPor=dataDocumento
2
https://dadosabertos.camara.leg.br/api/v2/deputados/204554/despesas?ano=2019&pagina=2&itens=100&ordem=ASC&ordenarPor=dataDocumento
3
https://dadosabertos.camara.leg.br/api/v2/deputados/204554/despesas?ano=2019&pagina=3&itens=100&ordem=ASC&ordenarPor=dataDocumento
1
https://dadosabertos.camara.leg.br/api/v2/deputados/204521/despesas?ano=2019&pagina=1&itens=100&ordem=ASC&ordenarPor=dataDocumento
2
https://dadosabertos.camara.leg.br/api/v2/deputados/204521/despesas?ano=2019&pagina=2&itens=100&ordem=ASC&ordenarPor=dataDocumento
3
https://dadosabertos.camara.leg.br/api/v2/deputados/204521/despesas?ano=2019&pagina=3&itens=100&ordem=ASC&ordenarPor=dataDocumento
4
https://dadosabertos.camara.leg.br/api/v2/deputados/204521/despesas?ano=2019&pagina=4&itens=100&ordem=ASC&ordenarPor=dataDocumento
1
https://dadosabertos.camara.leg.br/api/v2/deputados/204379/despesas

In [46]:
despesas.drop(columns=['ano','codLote','parcela','urlDocumento','valorLiquido'],inplace=True)

In [47]:
despesas.dtypes

cnpjCpfFornecedor            object
codDocumento                  int64
codTipoDocumento              int64
dataDocumento        datetime64[ns]
mes                           int64
nomeFornecedor               object
numDocumento                 object
numRessarcimento             object
tipoDespesa                  object
tipoDocumento                object
valorDocumento              float64
valorGlosa                  float64
dep_id                        int64
dep_nome                     object
dep_partido                  object
dep_uf                       object
dtype: object

In [62]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].count().sort_values(ascending=False)

nomeFornecedor
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA                          25
FRATELLI POSTO DE COMBUSTIVEIS LTDA                             13
JMF COMERCIO DE COMBUSTIVEIS LTDA - CABULA                       7
MM COMERCIAL DE COMBUSTIVEIS LTDA                                7
POSTO E RESTAURANTE SÃO PAULO LTDA                               5
AUTO POSTO AEROPORTO LTDA                                        5
3 VIAS COMERCIO DE DERIVADOS DE PETROLEO                         3
AUTO POSTO NOVO CENTRO LTDA                                      3
NENENS CHOPP COM VAREJ DE COMB IND E AGROP LTDA                  3
POSTO KALILANDIA LTDA                                            3
PETROMERC DERIVADOS DE PETROLEO LTDA.                            2
OCEANO BLUE COMERCIO DE COMBUSTIVEIS LTDA                        2
MIGUEL AUTO POSTO LTDA                                           2
POSTO DE COMBUSTIVEIS RODOVIARIA LTDA                            2
BRASAL COMBUSTIVEIS LTDA                       

In [67]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].sum().sort_values(ascending=False)

nomeFornecedor
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA                          4409.46
FRATELLI POSTO DE COMBUSTIVEIS LTDA                             3064.42
JMF COMERCIO DE COMBUSTIVEIS LTDA - CABULA                      1085.39
AUTO POSTO AEROPORTO LTDA                                        919.87
POSTO E RESTAURANTE SÃO PAULO LTDA                               856.03
POSTO KALILANDIA LTDA                                            600.79
MM COMERCIAL DE COMBUSTIVEIS LTDA                                550.00
3 VIAS COMERCIO DE DERIVADOS DE PETROLEO                         490.09
POSTO DE COMBUSTIVEIS JAGUARIB                                   389.80
POSTO DISBRAVE IMPERIAL LTDA                                     350.00
POSTO DE COMBUSTIVEIS RODOVIARIA LTDA                            343.58
POSTO ROSARIO LTDA                                               326.95
MIGUEL AUTO POSTO LTDA                                           326.05
BRASPETRO (QNM) COMERCIO DE COMBUSTIVEIS LTDA    

In [64]:
despesas.loc[(despesas['nomeFornecedor'].str.contains('CASCOL COMBUSTIVEIS'))]

,cnpjCpfFornecedor,codDocumento,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,tipoDespesa,tipoDocumento,valorDocumento,valorGlosa,dep_id,dep_nome,dep_partido,dep_uf
47,00306597007614,6778439,4,2019-03-12,3,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,349388,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,100.00,0.0,204554,ABÍLIO SANTANA,PL,BA
67,00306597007614,6787455,4,2019-03-20,3,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,357064,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,254.15,0.0,204554,ABÍLIO SANTANA,PL,BA
64,00306597007614,6786329,4,2019-03-19,3,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,356755,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,150.00,0.0,204521,ABOU ANNI,PSL,SP
20,00306597007614,6850805,4,2019-06-04,6,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,430214,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,200.00,0.0,204521,ABOU ANNI,PSL,SP
5,00306597003112,6773207,4,2019-02-02,2,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,218530,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,150.00,0.0,204379,ACÁCIO FAVACHO,PROS,AP
7,00306597003112,6773217,4,2019-02-07,2,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,220253,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,50.00,0.0,204379,ACÁCIO FAVACHO,PROS,AP
11,00306597007100,6773234,4,2019-02-13,2,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,437150,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,50.00,0.0,204379,ACÁCIO FAVACHO,PROS,AP
13,00306597003112,6773238,4,2019-02-14,2,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,222913,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,160.01,0.0,204379,ACÁCIO FAVACHO,PROS,AP
14,00306597008181,6773190,4,2019-02-14,2,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,561746,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,100.00,0.0,204379,ACÁCIO FAVACHO,PROS,AP
34,00306597008181,6795458,4,2019-03-09,3,CASCOL COMBUSTIVEIS PARA VEICULOS LTDA,578395,,COMBUSTÍVEIS E LUBRIFICANTES.,Nota Fiscal Eletrônica,120.00,0.0,204379,ACÁCIO FAVACHO,PROS,AP


In [13]:
despesas[(despesas['cnpjCpfFornecedor']=="17895646000187")].head()

,ano,cnpjCpfFornecedor,codDocumento,codLote,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,...,tipoDespesa,tipoDocumento,urlDocumento,valorDocumento,valorGlosa,valorLiquido,dep_id,dep_nome,dep_partido,dep_uf
54,2019,17895646000187,6787327,1576933,1,2019-03-12,3,UBER DO BRASIL TECNOLOGIA LTDA,s/n,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,57.80,0.0,57.80,204521,ABOU ANNI,PSL,SP
70,2019,17895646000187,6787329,1576933,1,2019-03-19,3,UBER DO BRASIL TECNOLOGIA LTDA,S/N,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,41.46,0.0,41.46,204521,ABOU ANNI,PSL,SP
73,2019,17895646000187,6799466,1581867,1,2019-03-21,4,UBER DO BRASIL TECNOLOGIA LTDA,21032019,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,12.30,0.0,12.30,204521,ABOU ANNI,PSL,SP
74,2019,17895646000187,6799370,1581851,1,2019-03-21,4,UBER DO BRASIL TECNOLOGIA LTDA.,212019,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,35.79,0.0,35.79,204521,ABOU ANNI,PSL,SP
84,2019,17895646000187,6799462,1581867,1,2019-03-26,4,UBER DO BRASIL TECNOLOGIA LTDA,26032019,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,69.85,0.0,69.85,204521,ABOU ANNI,PSL,SP
0,2019,17895646000187,6824505,1592348,1,2019-04-02,5,UBER DO BRASIL TECNOLOGIA LTDA,sn,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,40.09,0.0,40.09,204521,ABOU ANNI,PSL,SP
16,2019,17895646000187,6808999,1585875,1,2019-04-09,4,UBER DO BRASIL TECNOLOGIA LTDA,09042019,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,11.63,0.0,11.63,204521,ABOU ANNI,PSL,SP
37,2019,17895646000187,6824512,1592348,1,2019-04-18,5,UBER DO BRASIL TECNOLOGIA LTDA,sn,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,29.43,0.0,29.43,204521,ABOU ANNI,PSL,SP
42,2019,17895646000187,6824499,1592348,1,2019-04-23,4,UBER DO BRASIL TECNOLOGIA LTDA,sn,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,28.32,0.0,28.32,204521,ABOU ANNI,PSL,SP
49,2019,17895646000187,6824519,1592348,1,2019-04-25,4,UBER DO BRASIL TECNOLOGIA LTDA,sn,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,17.61,0.0,17.61,204521,ABOU ANNI,PSL,SP


In [11]:
despesas.dtypes

ano                    int64
cnpjCpfFornecedor     object
codDocumento           int64
codLote                int64
codTipoDocumento       int64
dataDocumento         object
mes                    int64
nomeFornecedor        object
numDocumento          object
numRessarcimento      object
parcela                int64
tipoDespesa           object
tipoDocumento         object
urlDocumento          object
valorDocumento       float64
valorGlosa           float64
valorLiquido         float64
dep_id                 int64
dep_nome              object
dep_partido           object
dep_uf                object
dtype: object

In [16]:
despesas.to_excel('despesas.xlsx')

In [ ]:
despesas.groupby(by=['dep_id']).count()

In [41]:
despesas.count()

ano                  697
cnpjCpfFornecedor    697
codDocumento         697
codLote              697
codTipoDocumento     697
dataDocumento        672
mes                  697
nomeFornecedor       697
numDocumento         697
numRessarcimento     697
parcela              697
tipoDespesa          697
tipoDocumento        697
urlDocumento         697
valorDocumento       697
valorGlosa           697
valorLiquido         697
dep_id               697
dep_nome             697
dep_partido          697
dep_uf               697
dtype: int64

In [48]:
despesas.profile_report(style={'full_width':True})

In [15]:
despesas['dataDocumento'] = pd.to_datetime(despesas['dataDocumento'],infer_datetime_format=True)

In [27]:
despesas['dataDocumento']

0           NaT
1           NaT
2           NaT
3           NaT
4           NaT
5    2019-02-01
6    2019-02-02
7    2019-02-03
8    2019-02-06
9    2019-02-07
10   2019-02-07
11   2019-02-07
12   2019-02-08
13   2019-02-12
14   2019-02-12
15   2019-02-12
16   2019-02-13
17   2019-02-13
18   2019-02-14
19   2019-02-14
20   2019-02-14
21   2019-02-14
22   2019-02-15
23   2019-02-16
24   2019-02-16
25   2019-02-18
26   2019-02-18
27   2019-02-18
28   2019-02-19
29   2019-02-19
        ...    
72   2019-04-14
73   2019-04-14
74   2019-04-14
75   2019-04-15
76   2019-04-15
77   2019-04-17
78   2019-04-17
79   2019-04-19
80   2019-04-23
81   2019-04-23
82   2019-04-25
83   2019-04-29
84   2019-04-30
85   2019-05-07
86   2019-05-07
87   2019-05-09
88   2019-05-09
89   2019-05-09
90   2019-05-09
91   2019-05-10
92   2019-05-12
93   2019-05-14
94   2019-05-14
95   2019-05-16
96   2019-05-16
97   2019-05-20
98   2019-05-21
99   2019-05-22
0    2019-05-23
1    2019-05-23
Name: dataDocumento, Len

In [36]:
despesas.groupby(by=["dataDocumento"])['tipoDespesa'].count()

dataDocumento
2019-02-01     2
2019-02-02     2
2019-02-03     1
2019-02-04     1
2019-02-05     4
2019-02-06     3
2019-02-07    11
2019-02-08     4
2019-02-09     6
2019-02-10     1
2019-02-11     2
2019-02-12     7
2019-02-13     9
2019-02-14    15
2019-02-15     2
2019-02-16     3
2019-02-17     3
2019-02-18    17
2019-02-19     6
2019-02-20    12
2019-02-21    11
2019-02-22     7
2019-02-23     2
2019-02-24     4
2019-02-25     6
2019-02-26    13
2019-02-27     4
2019-02-28     4
2019-03-01     6
2019-03-02     3
              ..
2019-05-26     1
2019-05-27     9
2019-05-28     7
2019-05-29     4
2019-05-30     3
2019-05-31     6
2019-06-01     4
2019-06-03     2
2019-06-04     5
2019-06-05     1
2019-06-06     4
2019-06-07     1
2019-06-08     1
2019-06-11     2
2019-06-12     4
2019-06-13     5
2019-06-14     1
2019-06-17     2
2019-06-18     3
2019-06-19     6
2019-06-25     1
2019-06-26     3
2019-06-27     2
2019-06-28     3
2019-06-29     1
2019-07-01     1
2019-07-02     2


In [29]:
despesas.head(n=15)

,ano,cnpjCpfFornecedor,codDocumento,codLote,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,...,tipoDespesa,tipoDocumento,urlDocumento,valorDocumento,valorGlosa,valorLiquido,dep_id,dep_nome,dep_partido,dep_uf
0,2019,00000000000006,0,0,0,NaT,2,RAMAL,6836040,0,...,TELEFONIA,Nota Fiscal,,75.66,0.0,75.66,204554,ABÍLIO SANTANA,PL,BA
1,2019,00000000000006,0,0,0,NaT,3,RAMAL,6858322,0,...,TELEFONIA,Nota Fiscal,,10.44,0.0,10.44,204554,ABÍLIO SANTANA,PL,BA
2,2019,00000000000006,0,0,0,NaT,4,RAMAL,6860198,0,...,TELEFONIA,Nota Fiscal,,14.33,0.0,14.33,204554,ABÍLIO SANTANA,PL,BA
3,2019,00000000000006,0,0,0,NaT,5,RAMAL,6860309,0,...,TELEFONIA,Nota Fiscal,,16.89,0.0,16.89,204554,ABÍLIO SANTANA,PL,BA
4,2019,00000000000006,0,0,0,NaT,6,RAMAL,6867342,0,...,TELEFONIA,Nota Fiscal,,22.07,0.0,22.07,204554,ABÍLIO SANTANA,PL,BA
5,2019,02575829000148,6759637,1566457,0,2019-02-01,2,AVIANCA BRASIL,2472443863483,,...,PASSAGENS AÉREAS,Nota Fiscal,,2231.97,0.0,2231.97,204554,ABÍLIO SANTANA,PL,BA
6,2019,10348318000979,6758921,1566200,4,2019-02-02,2,WINDSOR ADM. DE HOTEIS E SERVIÇOS LTDA,78750,,...,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",Nota Fiscal Eletrônica,,1495.00,0.0,1495.00,204554,ABÍLIO SANTANA,PL,BA
7,2019,02575829000148,6759639,1566457,0,2019-02-03,2,AVIANCA BRASIL,2472444244801,,...,PASSAGENS AÉREAS,Nota Fiscal,,765.38,0.0,765.38,204554,ABÍLIO SANTANA,PL,BA
8,2019,69797870120,6762195,1567360,1,2019-02-06,2,ALEXANDRE,SEM NUMERO,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,25.00,0.0,25.00,204554,ABÍLIO SANTANA,PL,BA
9,2019,01654521108,6762169,1567360,1,2019-02-07,2,WILKER,sem numero,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,20.00,0.0,20.00,204554,ABÍLIO SANTANA,PL,BA


### Gráficos